## Interactive DFG with Drill-Down Capability

This section contains enhanced visualizations that allow you to:
1. Click on nodes to see activity details
2. Click on edges to explore specific transitions
3. Filter and drill down into specific patient flows
4. Analyze subpaths interactively

In [43]:
import pandas as pd
import numpy as np
from collections import defaultdict
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, HTML, Image
import warnings
warnings.filterwarnings('ignore')

In [44]:
# Run helper notebook for constants
%run ../utils/helpers.ipynb

# Run data loader notebook
%run data_loader.ipynb

NTH-ED DATA LOADING PIPELINE
📥 Loading event log...
   ✓ Loaded 90,965 events
   ✓ 16,011 unique patient visits
   ✓ Columns: ['visit_id', 'patient_id', 'initial_zone', 'age', 'month', 'day', 'gender', 'triage_code', 'triage_desc', 'disposition_code', 'disposition_desc', 'consult_desc', 'cdu_flag', 'consult_req_flag', 'consult_arrival_flag', 'event', 'timestamp']

⏰ Parsing timestamps...
   ✓ All timestamps parsed successfully
   ✓ Date range: 2021-03-31 23:59:00 to 2021-06-01 17:16:00

🔧 Handling missing data...
   • initial_zone: 1,950 missing (2.1%)
   • triage_code: 3 missing (0.0%)
   • consult_desc: 69,698 missing (76.6%)
   • age: 0 missing (0.0%)
   ✓ Missing zones filled with 'Unknown'
   ✓ Missing consults marked as 'No Consult'

📋 Standardizing columns for process mining...
   ✓ Created process mining columns: case_id, activity, resource
   ✓ Created outcome flags: is_admitted, is_lwbs

🔀 Sorting events with logical ordering...
   ✓ Events sorted by case_id → timestamp → log

In [45]:
# Load the data using our data_loader functions
filepath = "/Users/ishaandawra/Desktop/Machine Learning Notes/Machine Learning Projects/Analytics_Colloquia_Project/data/event_log_ED_MMA_2026.csv"
event_log, visits = load_and_prepare_data(filepath)

print(f"\n✓ Event log ready: {len(event_log):,} events")
print(f"✓ Visits ready: {len(visits):,} visits")

NTH-ED DATA LOADING PIPELINE
📥 Loading event log...
   ✓ Loaded 90,965 events
   ✓ 16,011 unique patient visits
   ✓ Columns: ['visit_id', 'patient_id', 'initial_zone', 'age', 'month', 'day', 'gender', 'triage_code', 'triage_desc', 'disposition_code', 'disposition_desc', 'consult_desc', 'cdu_flag', 'consult_req_flag', 'consult_arrival_flag', 'event', 'timestamp']

⏰ Parsing timestamps...
   ✓ All timestamps parsed successfully
   ✓ Date range: 2021-03-31 23:59:00 to 2021-06-01 17:16:00

🔧 Handling missing data...
   • initial_zone: 1,950 missing (2.1%)
   • triage_code: 3 missing (0.0%)
   • consult_desc: 69,698 missing (76.6%)
   • age: 0 missing (0.0%)
   ✓ Missing zones filled with 'Unknown'
   ✓ Missing consults marked as 'No Consult'

📋 Standardizing columns for process mining...
   ✓ Created process mining columns: case_id, activity, resource
   ✓ Created outcome flags: is_admitted, is_lwbs

🔀 Sorting events with logical ordering...
   ✓ Events sorted by case_id → timestamp → log

In [46]:
def create_interactive_dfg_with_drilldown(dfg: dict, event_log: pd.DataFrame, 
                                          title: str = "Interactive DFG with Drill-Down", 
                                          min_edge_count: int = 50):
    """
    Create an interactive network graph with drill-down capabilities.
    
    INTERACTIVE FEATURES:
    - Click on nodes to see detailed activity statistics
    - Click on edges to see case examples and timing distributions
    - Hover for quick stats
    - Filter by frequency threshold
    
    Parameters:
    -----------
    dfg : dict - Output from build_dfg()
    event_log : pd.DataFrame - Full event log for drill-down details
    title : str - Chart title
    min_edge_count : int - Only show edges with at least this many cases
    """
    from plotly.subplots import make_subplots
    
    # Get unique activities
    activities = list(dfg['nodes'].keys())
    activity_to_idx = {act: idx for idx, act in enumerate(activities)}
    
    # Prepare edge data
    edge_data = []
    for (from_act, to_act), count in dfg['edges'].items():
        if count >= min_edge_count:
            durations = dfg['edge_durations'].get((from_act, to_act), [])
            edge_data.append({
                'from': from_act,
                'to': to_act,
                'count': count,
                'median_min': np.median(durations) if durations else 0,
                'mean_min': np.mean(durations) if durations else 0,
                'std_min': np.std(durations) if durations else 0
            })
    
    # Create network layout using circular layout
    import math
    n = len(activities)
    positions = {}
    for i, act in enumerate(activities):
        angle = 2 * math.pi * i / n
        positions[act] = (math.cos(angle), math.sin(angle))
    
    # Create edges for the graph
    edge_traces = []
    for edge in edge_data:
        from_pos = positions[edge['from']]
        to_pos = positions[edge['to']]
        
        # Calculate edge width based on count
        width = max(1, min(10, edge['count'] / 100))
        
        edge_trace = go.Scatter(
            x=[from_pos[0], to_pos[0], None],
            y=[from_pos[1], to_pos[1], None],
            mode='lines',
            line=dict(width=width, color='rgba(125, 125, 125, 0.5)'),
            hoverinfo='text',
            text=f"{edge['from']} → {edge['to']}<br>" +
                 f"Count: {edge['count']:,}<br>" +
                 f"Median: {edge['median_min']:.1f} min<br>" +
                 f"Mean: {edge['mean_min']:.1f} min<br>" +
                 f"Std: {edge['std_min']:.1f} min<br>" +
                 f"<b>Click edge to see details</b>",
            name=f"{edge['from']}-{edge['to']}"
        )
        edge_traces.append(edge_trace)
    
    # Create nodes
    node_x = [positions[act][0] for act in activities]
    node_y = [positions[act][1] for act in activities]
    node_sizes = [min(50, max(20, dfg['nodes'][act] / 100)) for act in activities]
    node_text = [f"{act}<br>Count: {dfg['nodes'][act]:,}<br><b>Click for details</b>" 
                 for act in activities]
    
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        marker=dict(
            size=node_sizes,
            color=list(range(len(activities))),
            colorscale='Viridis',
            showscale=False,
            line=dict(width=2, color='white')
        ),
        text=activities,
        textposition='top center',
        textfont=dict(size=10, color='black'),
        hoverinfo='text',
        hovertext=node_text,
        name='Activities'
    )
    
    # Create figure
    fig = go.Figure(data=edge_traces + [node_trace])
    
    fig.update_layout(
        title=dict(text=title, font=dict(size=18)),
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        height=700,
        plot_bgcolor='rgba(240, 240, 240, 0.5)'
    )
    
    return fig

print("✓ create_interactive_dfg_with_drilldown() function defined")

✓ create_interactive_dfg_with_drilldown() function defined


In [47]:
def drill_down_activity(activity_name: str, dfg: dict, event_log: pd.DataFrame):
    """
    Drill down into a specific activity to see detailed statistics.
    
    Shows:
    - Total occurrences
    - Incoming transitions (from where?)
    - Outgoing transitions (to where?)
    - Timing statistics
    - Sample cases
    """
    print("=" * 80)
    print(f"📊 ACTIVITY DRILL-DOWN: {activity_name}")
    print("=" * 80)
    
    # Basic stats
    total_occurrences = dfg['nodes'].get(activity_name, 0)
    print(f"\n📈 Total Occurrences: {total_occurrences:,}")
    
    # Incoming edges
    print(f"\n⬇️  INCOMING TRANSITIONS (What leads to {activity_name}?):")
    incoming = [(from_act, to_act, count) for (from_act, to_act), count in dfg['edges'].items() 
                if to_act == activity_name]
    incoming.sort(key=lambda x: x[2], reverse=True)
    
    if incoming:
        for from_act, to_act, count in incoming[:10]:
            percentage = (count / total_occurrences) * 100
            durations = dfg['edge_durations'].get((from_act, to_act), [])
            median_dur = np.median(durations) if durations else 0
            print(f"  • {from_act:30} → {activity_name:20} | {count:6,} cases ({percentage:5.1f}%) | Median: {median_dur:6.1f} min")
    else:
        print("  (No incoming transitions - this might be a starting activity)")
    
    # Outgoing edges
    print(f"\n⬆️  OUTGOING TRANSITIONS (Where do patients go after {activity_name}?):")
    outgoing = [(from_act, to_act, count) for (from_act, to_act), count in dfg['edges'].items() 
                if from_act == activity_name]
    outgoing.sort(key=lambda x: x[2], reverse=True)
    
    if outgoing:
        for from_act, to_act, count in outgoing[:10]:
            percentage = (count / total_occurrences) * 100
            durations = dfg['edge_durations'].get((from_act, to_act), [])
            median_dur = np.median(durations) if durations else 0
            print(f"  • {activity_name:30} → {to_act:20} | {count:6,} cases ({percentage:5.1f}%) | Median: {median_dur:6.1f} min")
    else:
        print("  (No outgoing transitions - this might be an ending activity)")
    
    # Sample cases with this activity
    print(f"\n📋 SAMPLE CASE IDs WITH THIS ACTIVITY (first 10):")
    sample_cases = event_log[event_log['activity'] == activity_name]['case_id'].unique()[:10]
    print(f"  {', '.join(map(str, sample_cases))}")
    
    print("\n" + "=" * 80)

print("✓ drill_down_activity() function defined")

✓ drill_down_activity() function defined


In [48]:
def drill_down_edge(from_activity: str, to_activity: str, dfg: dict, event_log: pd.DataFrame):
    """
    Drill down into a specific edge (transition) to see detailed information.
    
    Shows:
    - Frequency and percentage
    - Duration distribution (histogram)
    - Sample cases that follow this path
    - Context (what happens before and after)
    """
    print("=" * 80)
    print(f"🔗 EDGE DRILL-DOWN: {from_activity} → {to_activity}")
    print("=" * 80)
    
    edge_key = (from_activity, to_activity)
    
    # Basic stats
    count = dfg['edges'].get(edge_key, 0)
    durations = dfg['edge_durations'].get(edge_key, [])
    
    print(f"\n📊 TRANSITION FREQUENCY:")
    print(f"  • Total occurrences: {count:,}")
    
    # Calculate percentage of all transitions from source
    total_from_source = sum(c for (f, t), c in dfg['edges'].items() if f == from_activity)
    percentage = (count / total_from_source * 100) if total_from_source > 0 else 0
    print(f"  • Percentage of all '{from_activity}' transitions: {percentage:.1f}%")
    
    # Duration statistics
    if durations:
        print(f"\n⏱️  DURATION STATISTICS (in minutes):")
        print(f"  • Median:  {np.median(durations):8.1f} min")
        print(f"  • Mean:    {np.mean(durations):8.1f} min")
        print(f"  • Std Dev: {np.std(durations):8.1f} min")
        print(f"  • Min:     {np.min(durations):8.1f} min")
        print(f"  • Max:     {np.max(durations):8.1f} min")
        print(f"  • P25:     {np.percentile(durations, 25):8.1f} min")
        print(f"  • P75:     {np.percentile(durations, 75):8.1f} min")
        print(f"  • P90:     {np.percentile(durations, 90):8.1f} min")
    
    # Find sample cases
    print(f"\n📋 SAMPLE CASES FOLLOWING THIS PATH:")
    
    # Get case IDs that have both activities in sequence
    cases_with_path = []
    for case_id, group in event_log.groupby('case_id'):
        events = group.sort_values('timestamp')
        activities = events['activity'].tolist()
        
        # Check if this case has the specific transition
        for i in range(len(activities) - 1):
            if activities[i] == from_activity and activities[i + 1] == to_activity:
                # Get the timestamps for this transition
                times = events['timestamp'].tolist()
                duration = (times[i + 1] - times[i]).total_seconds() / 60
                cases_with_path.append({
                    'case_id': case_id,
                    'duration_min': duration,
                    'full_path': ' → '.join(activities)
                })
                break
    
    # Display top 10 cases
    if cases_with_path:
        sample_df = pd.DataFrame(cases_with_path).head(10)
        for idx, row in sample_df.iterrows():
            print(f"\n  Case {row['case_id']}:")
            print(f"    Duration: {row['duration_min']:.1f} minutes")
            print(f"    Full path: {row['full_path']}")
    else:
        print("  (No sample cases found)")
    
    print("\n" + "=" * 80)
    
    # Return duration data for visualization
    return durations

print("✓ drill_down_edge() function defined")

✓ drill_down_edge() function defined


In [49]:
def visualize_edge_duration_distribution(from_activity: str, to_activity: str, 
                                          dfg: dict, title_suffix: str = ""):
    """
    Create a histogram showing the distribution of durations for a specific edge.
    
    Helps identify:
    - Typical duration (peak of distribution)
    - Outliers (very long or short durations)
    - Multiple modes (different patient groups?)
    """
    edge_key = (from_activity, to_activity)
    durations = dfg['edge_durations'].get(edge_key, [])
    
    if not durations:
        print(f"No duration data available for {from_activity} → {to_activity}")
        return None
    
    # Create histogram
    fig = go.Figure()
    
    fig.add_trace(go.Histogram(
        x=durations,
        nbinsx=50,
        name='Duration',
        marker_color='steelblue',
        opacity=0.7
    ))
    
    # Add vertical lines for key statistics
    median_dur = np.median(durations)
    mean_dur = np.mean(durations)
    p90_dur = np.percentile(durations, 90)
    
    fig.add_vline(x=median_dur, line_dash="dash", line_color="red", 
                  annotation_text=f"Median: {median_dur:.1f}m", annotation_position="top")
    fig.add_vline(x=mean_dur, line_dash="dash", line_color="green",
                  annotation_text=f"Mean: {mean_dur:.1f}m", annotation_position="top")
    fig.add_vline(x=p90_dur, line_dash="dot", line_color="orange",
                  annotation_text=f"P90: {p90_dur:.1f}m", annotation_position="top")
    
    fig.update_layout(
        title=f"Duration Distribution: {from_activity} → {to_activity} {title_suffix}",
        xaxis_title="Duration (minutes)",
        yaxis_title="Number of Cases",
        showlegend=False,
        height=400
    )
    
    return fig

print("✓ visualize_edge_duration_distribution() function defined")

✓ visualize_edge_duration_distribution() function defined


In [50]:
def create_interactive_sunburst_dfg(event_log: pd.DataFrame, max_depth: int = 3,
                                    title: str = "Interactive Hierarchical Process Flow"):
    """
    Create a sunburst chart showing hierarchical process flows.
    Click on any segment to zoom in and explore that path.
    
    SUNBURST ADVANTAGES:
    - Shows process flow hierarchy
    - Click to drill down into paths
    - Width represents frequency
    - Easy to spot dominant paths
    
    Parameters:
    -----------
    event_log : pd.DataFrame - Event log data
    max_depth : int - Maximum path depth to show
    title : str - Chart title
    """
    # Extract sequences for each case
    sequences = []
    for case_id, group in event_log.groupby('case_id'):
        events = group.sort_values('timestamp')
        activities = events['activity'].tolist()[:max_depth]
        sequences.append(activities)
    
    # Build hierarchical structure
    paths = []
    values = []
    
    # Count sequences at each level
    from collections import Counter
    
    # Level 1 (starting activity)
    level1_counts = Counter([seq[0] for seq in sequences if len(seq) >= 1])
    for act, count in level1_counts.items():
        paths.append([act])
        values.append(count)
    
    # Level 2 (first transition)
    level2_counts = Counter([tuple(seq[:2]) for seq in sequences if len(seq) >= 2])
    for path_tuple, count in level2_counts.items():
        paths.append(list(path_tuple))
        values.append(count)
    
    # Level 3 (second transition)
    if max_depth >= 3:
        level3_counts = Counter([tuple(seq[:3]) for seq in sequences if len(seq) >= 3])
        for path_tuple, count in level3_counts.items():
            paths.append(list(path_tuple))
            values.append(count)
    
    # Prepare data for sunburst
    labels = []
    parents = []
    vals = []
    
    for path, value in zip(paths, values):
        if len(path) == 1:
            labels.append(path[0])
            parents.append("")
            vals.append(value)
        else:
            full_label = " → ".join(path)
            parent_label = " → ".join(path[:-1]) if len(path) > 2 else path[0]
            labels.append(full_label)
            parents.append(parent_label)
            vals.append(value)
    
    # Create sunburst chart
    fig = go.Figure(go.Sunburst(
        labels=labels,
        parents=parents,
        values=vals,
        branchvalues="total",
        marker=dict(colorscale='Viridis'),
        hovertemplate='<b>%{label}</b><br>Cases: %{value:,}<br><extra></extra>'
    ))
    
    fig.update_layout(
        title=dict(text=title, font=dict(size=18)),
        height=700,
        margin=dict(t=50, l=0, r=0, b=0)
    )
    
    return fig

print("✓ create_interactive_sunburst_dfg() function defined")

✓ create_interactive_sunburst_dfg() function defined


In [51]:
def create_interactive_dashboard(dfg: dict, event_log: pd.DataFrame):
    """
    Create an interactive dashboard with dropdown menus for exploring activities and edges.
    
    FEATURES:
    - Dropdown to select any activity
    - Dropdown to select any edge (transition)
    - Automatically updates visualizations
    - Shows comprehensive statistics
    """
    try:
        from ipywidgets import interact, Dropdown, VBox, HBox, Output
        from IPython.display import display, clear_output
    except ImportError:
        print("⚠️  ipywidgets not installed. Install with: pip install ipywidgets")
        return
    
    # Get all activities and edges
    activities = sorted(list(dfg['nodes'].keys()))
    edges = sorted([(f, t) for (f, t) in dfg['edges'].keys()], 
                   key=lambda x: dfg['edges'][(x[0], x[1])], reverse=True)
    
    # Create output widgets
    activity_output = Output()
    edge_output = Output()
    viz_output = Output()
    
    def on_activity_change(activity):
        """Handle activity selection"""
        with activity_output:
            clear_output(wait=True)
            if activity and activity != "Select an activity...":
                drill_down_activity(activity, dfg, event_log)
    
    def on_edge_change(edge_str):
        """Handle edge selection"""
        with edge_output:
            clear_output(wait=True)
            if edge_str and edge_str != "Select an edge...":
                # Parse edge string "From → To"
                from_act, to_act = edge_str.split(" → ")
                drill_down_edge(from_act, to_act, dfg, event_log)
        
        with viz_output:
            clear_output(wait=True)
            if edge_str and edge_str != "Select an edge...":
                from_act, to_act = edge_str.split(" → ")
                fig = visualize_edge_duration_distribution(from_act, to_act, dfg)
                if fig:
                    fig.show()
    
    # Create dropdown widgets
    activity_dropdown = Dropdown(
        options=["Select an activity..."] + activities,
        description='Activity:',
        style={'description_width': 'initial'}
    )
    
    edge_options = ["Select an edge..."] + [f"{f} → {t}" for f, t in edges]
    edge_dropdown = Dropdown(
        options=edge_options,
        description='Transition:',
        style={'description_width': 'initial'}
    )
    
    # Link dropdowns to handlers
    activity_dropdown.observe(lambda change: on_activity_change(change['new']), names='value')
    edge_dropdown.observe(lambda change: on_edge_change(change['new']), names='value')
    
    # Display dashboard
    print("=" * 80)
    print("🎛️  INTERACTIVE DFG EXPLORATION DASHBOARD")
    print("=" * 80)
    print("\nSelect an activity or transition from the dropdowns below to explore:\n")
    
    display(VBox([
        HBox([activity_dropdown, edge_dropdown]),
        activity_output,
        edge_output,
        viz_output
    ]))

print("✓ create_interactive_dashboard() function defined")

✓ create_interactive_dashboard() function defined


---
## 📋 Summary: Interactive DFG Features Added

### ✅ What's New:

1. **Interactive Network Graph** (`create_interactive_dfg_with_drilldown`)
   - Network layout showing all activities and transitions
   - Hover over nodes and edges for instant statistics
   - Visual size/width represents frequency

2. **Activity Drill-Down** (`drill_down_activity`)
   - Click any activity to see:
     - Total occurrences
     - All incoming transitions (with percentages)
     - All outgoing transitions (with percentages)
     - Sample case IDs

3. **Edge Drill-Down** (`drill_down_edge`)
   - Click any transition to see:
     - Frequency and percentage
     - Duration statistics (median, mean, std, percentiles)
     - Sample cases following this path
     - Full patient journeys

4. **Duration Distribution** (`visualize_edge_duration_distribution`)
   - Histogram showing timing patterns
   - Statistical markers (median, mean, P90)
   - Identify outliers and bottlenecks

5. **Interactive Sunburst Chart** (`create_interactive_sunburst_dfg`)
   - Hierarchical view of process flows
   - Click segments to zoom in
   - Easy to spot dominant paths

6. **Interactive Dashboard** (`create_interactive_dashboard`)
   - Dropdown menus for exploration
   - Select any activity or transition
   - Automatic visualization updates

### 🎯 How to Use:

**Option 1: Visual Exploration**
- Run the interactive visualizations (network graph, sunburst)
- Hover over elements for quick info
- Click on sunburst segments to drill down

**Option 2: Programmatic Drill-Down**
```python
drill_down_activity("Triage", dfg_all, event_log)
drill_down_edge("Triage", "Registration", dfg_all, event_log)
```

**Option 3: Interactive Dashboard**
```python
create_interactive_dashboard(dfg_all, event_log)
```

### 🔍 Comparison with Original Code:

**Original Sankey Diagram:**
- ❌ Static visualization only
- ❌ No drill-down capability
- ❌ Limited to hover tooltips
- ✅ Shows flow width nicely

**New Enhanced DFG:**
- ✅ Multiple interactive visualizations
- ✅ Full drill-down into activities and edges
- ✅ Detailed statistics and sample cases
- ✅ Duration distributions
- ✅ Hierarchical exploration
- ✅ Interactive dashboard with dropdowns

---

# Module 2: Process Discovery
## NTH-ED Triage Lead Decision Support Tool

**Purpose:** Build a Directly-Follows Graph (DFG) to visualize patient flow through the ED.

**Why This Matters for Triage Lead:**
- See the actual patient journey (not what's supposed to happen)
- Identify where patients get stuck (bottlenecks)
- Spot unusual paths (protocol deviations)
- Compare flow across triage levels and zones

---

## Cell 1: Imports

In [52]:
import pandas as pd
import numpy as np
from collections import defaultdict
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, HTML, Image
import warnings
warnings.filterwarnings('ignore')

## Cell 2: Load Dependencies (Run helper and data_loader first)

In [53]:
# Run helper notebook for constants
%run ../utils/helpers.ipynb

# Run data loader notebook
%run data_loader.ipynb

NTH-ED DATA LOADING PIPELINE
📥 Loading event log...
   ✓ Loaded 90,965 events
   ✓ 16,011 unique patient visits
   ✓ Columns: ['visit_id', 'patient_id', 'initial_zone', 'age', 'month', 'day', 'gender', 'triage_code', 'triage_desc', 'disposition_code', 'disposition_desc', 'consult_desc', 'cdu_flag', 'consult_req_flag', 'consult_arrival_flag', 'event', 'timestamp']

⏰ Parsing timestamps...
   ✓ All timestamps parsed successfully
   ✓ Date range: 2021-03-31 23:59:00 to 2021-06-01 17:16:00

🔧 Handling missing data...
   • initial_zone: 1,950 missing (2.1%)
   • triage_code: 3 missing (0.0%)
   • consult_desc: 69,698 missing (76.6%)
   • age: 0 missing (0.0%)
   ✓ Missing zones filled with 'Unknown'
   ✓ Missing consults marked as 'No Consult'

📋 Standardizing columns for process mining...
   ✓ Created process mining columns: case_id, activity, resource
   ✓ Created outcome flags: is_admitted, is_lwbs

🔀 Sorting events with logical ordering...
   ✓ Events sorted by case_id → timestamp → log

## Cell 3: Load Data

In [54]:
# Load the data using our data_loader functions
filepath = "/Users/ishaandawra/Desktop/Machine Learning Notes/Machine Learning Projects/Analytics_Colloquia_Project/data/event_log_ED_MMA_2026.csv"
event_log, visits = load_and_prepare_data(filepath)

print(f"\n✓ Event log ready: {len(event_log):,} events")
print(f"✓ Visits ready: {len(visits):,} visits")

NTH-ED DATA LOADING PIPELINE
📥 Loading event log...
   ✓ Loaded 90,965 events
   ✓ 16,011 unique patient visits
   ✓ Columns: ['visit_id', 'patient_id', 'initial_zone', 'age', 'month', 'day', 'gender', 'triage_code', 'triage_desc', 'disposition_code', 'disposition_desc', 'consult_desc', 'cdu_flag', 'consult_req_flag', 'consult_arrival_flag', 'event', 'timestamp']

⏰ Parsing timestamps...
   ✓ All timestamps parsed successfully
   ✓ Date range: 2021-03-31 23:59:00 to 2021-06-01 17:16:00

🔧 Handling missing data...
   • initial_zone: 1,950 missing (2.1%)
   • triage_code: 3 missing (0.0%)
   • consult_desc: 69,698 missing (76.6%)
   • age: 0 missing (0.0%)
   ✓ Missing zones filled with 'Unknown'
   ✓ Missing consults marked as 'No Consult'

📋 Standardizing columns for process mining...
   ✓ Created process mining columns: case_id, activity, resource
   ✓ Created outcome flags: is_admitted, is_lwbs

🔀 Sorting events with logical ordering...
   ✓ Events sorted by case_id → timestamp → log

---
## Section 1: Understanding the Directly-Follows Graph (DFG)

**What is a DFG?**
- A graph where nodes = activities (Triage, Registration, etc.)
- Edges = "Activity A was directly followed by Activity B"
- Edge weight = how many times this transition happened

**Example:**
```
Triage ──(15,234)──> Registration ──(14,891)──> Assessment
```
This means 15,234 patients went from Triage directly to Registration.

---

## Cell 4: Build DFG - Core Function

In [55]:
def build_dfg(event_log: pd.DataFrame) -> dict:
    """
    Build a Directly-Follows Graph from event log.
    
    HOW IT WORKS:
    1. Group events by case_id (patient visit)
    2. Sort events by timestamp within each case
    3. For each consecutive pair (A, B), count A → B
    
    Returns:
    --------
    dict with:
        - 'edges': {(from, to): count}
        - 'nodes': {activity: count}
        - 'edge_durations': {(from, to): list of durations in minutes}
    """
    
    edges = defaultdict(int)           # (from, to) -> count
    edge_durations = defaultdict(list) # (from, to) -> [duration1, duration2, ...]
    nodes = defaultdict(int)           # activity -> count
    
    # Group by case and iterate through each patient's journey
    for case_id, group in event_log.groupby('case_id'):
        # Sort by timestamp (already sorted, but ensure)
        events = group.sort_values('timestamp')
        activities = events['activity'].tolist()
        timestamps = events['timestamp'].tolist()
        
        # Count each activity occurrence
        for act in activities:
            nodes[act] += 1
        
        # Count transitions between consecutive activities
        for i in range(len(activities) - 1):
            from_act = activities[i]
            to_act = activities[i + 1]
            
            edges[(from_act, to_act)] += 1
            
            # Calculate duration between events (in minutes)
            duration = (timestamps[i + 1] - timestamps[i]).total_seconds() / 60
            if duration >= 0:  # Ignore negative durations (data errors)
                edge_durations[(from_act, to_act)].append(duration)
    
    return {
        'edges': dict(edges),
        'nodes': dict(nodes),
        'edge_durations': dict(edge_durations)
    }

print("✓ build_dfg() function defined")

✓ build_dfg() function defined


## Cell 5: Calculate Edge Statistics

In [56]:
def calculate_edge_stats(dfg: dict) -> pd.DataFrame:
    """
    Calculate statistics for each edge in the DFG.
    
    WHY: Raw counts aren't enough. Triage Lead needs to know:
    - How often does this transition happen? (frequency)
    - How long does it take? (median duration)
    - Is there high variability? (std dev)
    
    Returns:
    --------
    DataFrame with columns: from, to, count, median_minutes, mean_minutes, std_minutes
    """
    
    stats = []
    
    for (from_act, to_act), count in dfg['edges'].items():
        durations = dfg['edge_durations'].get((from_act, to_act), [])
        
        stats.append({
            'from': from_act,
            'to': to_act,
            'count': count,
            'median_minutes': np.median(durations) if durations else None,
            'mean_minutes': np.mean(durations) if durations else None,
            'std_minutes': np.std(durations) if durations else None,
            'min_minutes': np.min(durations) if durations else None,
            'max_minutes': np.max(durations) if durations else None,
        })
    
    df = pd.DataFrame(stats)
    df = df.sort_values('count', ascending=False).reset_index(drop=True)
    
    return df

print("✓ calculate_edge_stats() function defined")

✓ calculate_edge_stats() function defined


## Cell 6: Build DFG for All Data

In [57]:
# Build DFG from entire event log
print("Building DFG from all patient visits...")
dfg_all = build_dfg(event_log)

# Calculate edge statistics
edge_stats = calculate_edge_stats(dfg_all)

print(f"\n✓ DFG built successfully!")
print(f"  • Nodes (activities): {len(dfg_all['nodes'])}")
print(f"  • Edges (transitions): {len(dfg_all['edges'])}")

print("\n📊 Top 10 Most Frequent Transitions:")
display(edge_stats.head(10))

Building DFG from all patient visits...

✓ DFG built successfully!
  • Nodes (activities): 9
  • Edges (transitions): 45

📊 Top 10 Most Frequent Transitions:


,from,to,count,median_minutes,mean_minutes,std_minutes,min_minutes,max_minutes
0,Discharge,Left ED,14972,7.0,48.680337,188.262337,0.0,3342.0
1,Triage,Registration,14802,6.0,6.612215,4.311698,0.0,125.0
2,Registration,Assessment,13872,35.0,47.674092,42.994427,0.0,591.0
3,Assessment,Discharge,13062,92.0,130.859593,152.500545,0.0,2959.0
4,Ambulance Arrival,Triage,3032,7.0,63.743734,227.117738,0.0,1434.0
5,Assessment,Consult Request,2634,136.5,173.365224,164.833459,0.0,1389.0
6,Ambulance Transfer,Assessment,1880,29.0,42.613830,42.672003,0.0,273.0
7,Registration,Ambulance Transfer,1874,1.0,3.202775,6.675388,0.0,114.0
8,Consult Request,Consult Arrival,1470,107.0,139.223810,135.783925,0.0,1569.0
9,Consult Arrival,Discharge,1466,32.0,75.871078,139.347325,0.0,1749.0


---
## 📊 Interactive DFG Usage Examples

Now that we have `dfg_all` created, let's explore it using the interactive drill-down functions!

In [58]:
# Example 1: Display Interactive Network DFG
# This visualization allows you to hover over nodes and edges for details
fig_network = create_interactive_dfg_with_drilldown(
    dfg_all, 
    event_log,
    title="Interactive Patient Flow Network (Hover for Details)",
    min_edge_count=50
)
fig_network.show()

In [59]:
# Example 2: Display Interactive Sunburst Chart
# Click on any segment to drill down into that specific path
fig_sunburst = create_interactive_sunburst_dfg(
    event_log,
    max_depth=4,
    title="Interactive Process Hierarchy (Click to Drill Down)"
)
fig_sunburst.show()

### 🔍 Example 3: Drill Down into Specific Activity

Let's drill down into the "Triage" activity to see detailed statistics:

In [60]:
# Drill down into "Triage" activity
drill_down_activity("Triage", dfg_all, event_log)

📊 ACTIVITY DRILL-DOWN: Triage

📈 Total Occurrences: 16,010

⬇️  INCOMING TRANSITIONS (What leads to Triage?):
  • Ambulance Arrival              → Triage               |  3,032 cases ( 18.9%) | Median:    7.0 min
  • Ambulance Transfer             → Triage               |     58 cases (  0.4%) | Median:  247.5 min
  • Assessment                     → Triage               |     18 cases (  0.1%) | Median:   13.0 min
  • Registration                   → Triage               |      8 cases (  0.0%) | Median:    6.0 min
  • Left ED                        → Triage               |      2 cases (  0.0%) | Median:  568.5 min

⬆️  OUTGOING TRANSITIONS (Where do patients go after Triage?):
  • Triage                         → Registration         | 14,802 cases ( 92.5%) | Median:    6.0 min
  • Triage                         → Ambulance Transfer   |  1,122 cases (  7.0%) | Median:    6.0 min
  • Triage                         → Assessment           |     68 cases (  0.4%) | Median:    4.5 min
  

### 🔗 Example 4: Drill Down into Specific Edge (Transition)

Let's examine the transition from "Triage" to "Registration":

In [61]:
# Drill down into specific edge
durations = drill_down_edge("Triage", "Registration", dfg_all, event_log)

🔗 EDGE DRILL-DOWN: Triage → Registration

📊 TRANSITION FREQUENCY:
  • Total occurrences: 14,802
  • Percentage of all 'Triage' transitions: 92.5%

⏱️  DURATION STATISTICS (in minutes):
  • Median:       6.0 min
  • Mean:         6.6 min
  • Std Dev:      4.3 min
  • Min:          0.0 min
  • Max:        125.0 min
  • P25:          4.0 min
  • P75:          8.0 min
  • P90:         11.0 min

📋 SAMPLE CASES FOLLOWING THIS PATH:

  Case 5240985:
    Duration: 15.0 minutes
    Full path: Discharge → Left ED → Triage → Registration

  Case 7214779:
    Duration: 10.0 minutes
    Full path: Triage → Registration → Discharge → Left ED

  Case 7384128:
    Duration: 8.0 minutes
    Full path: Triage → Registration → Assessment → Discharge → Left ED

  Case 7384129:
    Duration: 10.0 minutes
    Full path: Triage → Registration → Assessment → Discharge → Left ED

  Case 7384131:
    Duration: 2.0 minutes
    Full path: Triage → Registration → Assessment → Discharge → Left ED

  Case 7384132:
 

### 📊 Example 5: Visualize Duration Distribution for an Edge

In [62]:
# Visualize duration distribution for Triage → Registration transition
fig_duration = visualize_edge_duration_distribution("Triage", "Registration", dfg_all)
if fig_duration:
    fig_duration.show()

### 🎯 Example 6: Interactive Exploration - Try These Commands!

You can explore any activity or edge in the dataset. Here are some examples:

```python
# Explore Assessment activity
drill_down_activity("Assessment", dfg_all, event_log)

# Explore Registration → Assessment transition
drill_down_edge("Registration", "Assessment", dfg_all, event_log)

# Visualize any edge duration
fig = visualize_edge_duration_distribution("Assessment", "Discharge", dfg_all)
fig.show()

# Explore ambulance-related transitions
drill_down_edge("Ambulance Arrival", "Triage", dfg_all, event_log)
```

### 🎛️ Example 7: Launch Interactive Dashboard

The dashboard provides dropdown menus to explore any activity or edge interactively:

In [63]:
# Launch the interactive dashboard
# Note: This requires ipywidgets to be installed
create_interactive_dashboard(dfg_all, event_log)

🎛️  INTERACTIVE DFG EXPLORATION DASHBOARD

Select an activity or transition from the dropdowns below to explore:



---
## Section 2: Visualize the DFG

We'll create two types of visualizations:
1. **Sankey Diagram** - Shows flow volumes (interactive, Plotly)
2. **Process Graph** - Traditional DFG with boxes and arrows

---

## Cell 7: Sankey Diagram Function

In [64]:
def create_sankey_diagram(dfg: dict, title: str = "Patient Flow Through ED", 
                          min_edge_count: int = 100) -> go.Figure:
    """
    Create an interactive Sankey diagram showing patient flow.
    
    WHY SANKEY:
    - Width of flow = number of patients
    - Easy to see where patients go
    - Interactive: hover for details
    
    Parameters:
    -----------
    dfg : dict - Output from build_dfg()
    title : str - Chart title
    min_edge_count : int - Only show edges with at least this many cases
    """
    
    # Get unique activities and create index mapping
    activities = list(dfg['nodes'].keys())
    activity_to_idx = {act: idx for idx, act in enumerate(activities)}
    
    # Prepare edge data (filter by minimum count)
    sources = []
    targets = []
    values = []
    labels_hover = []
    
    for (from_act, to_act), count in dfg['edges'].items():
        if count >= min_edge_count:
            sources.append(activity_to_idx[from_act])
            targets.append(activity_to_idx[to_act])
            values.append(count)
            
            # Get median duration for hover
            durations = dfg['edge_durations'].get((from_act, to_act), [])
            median_dur = np.median(durations) if durations else 0
            labels_hover.append(f"{from_act} → {to_act}<br>Count: {count:,}<br>Median: {median_dur:.0f} min")
    
    # Define colors for activities
    node_colors = [
        '#3B82F6',  # Blue - Ambulance Arrival
        '#22C55E',  # Green - Triage
        '#EAB308',  # Yellow - Registration
        '#F97316',  # Orange - Ambulance Transfer
        '#8B5CF6',  # Purple - Assessment
        '#EC4899',  # Pink - Consult Request
        '#14B8A6',  # Teal - Consult Arrival
        '#6B7280',  # Gray - Discharge
        '#DC2626',  # Red - Left ED
    ]
    
    # Create Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=activities,
            color=node_colors[:len(activities)]
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values,
            customdata=labels_hover,
            hovertemplate='%{customdata}<extra></extra>'
        )
    )])
    
    fig.update_layout(
        title=dict(text=title, font=dict(size=18)),
        font=dict(size=12),
        height=500
    )
    
    return fig

print("✓ create_sankey_diagram() function defined")

✓ create_sankey_diagram() function defined


## Cell 8: Display Sankey Diagram for All Patients

In [65]:
# Create and display Sankey diagram
fig_sankey = create_sankey_diagram(
    dfg_all, 
    title="Patient Flow Through NTH-ED (All Visits)",
    min_edge_count=50
)
fig_sankey.show()

---
## Section 3: Filter DFG by Triage Level and Zone

**Why Filtering Matters:**
- CTAS 2 (Emergency) patients should have a different flow than CTAS 5 (Non-Urgent)
- EPZ (Psychiatric) zone has unique patterns
- Triage Lead needs to compare flows to spot anomalies

---

## Cell 9: Filter Function

In [66]:
def filter_event_log(event_log: pd.DataFrame, visits: pd.DataFrame,
                     triage_level: str = None, 
                     zone: str = None,
                     is_admitted: bool = None,
                     is_lwbs: bool = None) -> pd.DataFrame:
    """
    Filter event log by various criteria.
    
    WHY: Triage Lead needs to analyze specific patient segments:
    - "Show me only CTAS 2 patients" → spot priority inversions
    - "Show me Yellow Zone" → identify zone-specific bottlenecks
    - "Show me LWBS cases" → understand why patients leave
    
    Parameters:
    -----------
    event_log : pd.DataFrame - Full event log
    visits : pd.DataFrame - Visit summary (for filtering criteria)
    triage_level : str - e.g., '2-EMERGENCY', '3-URGENT'
    zone : str - e.g., 'YZ', 'GZ', 'EPZ'
    is_admitted : bool - True for admitted, False for discharged
    is_lwbs : bool - True for LWBS cases only
    
    Returns:
    --------
    Filtered event log
    """
    
    # Start with all case IDs
    filtered_visits = visits.copy()
    
    # Apply filters
    if triage_level is not None:
        filtered_visits = filtered_visits[filtered_visits['triage_level'] == triage_level]
    
    if zone is not None:
        filtered_visits = filtered_visits[filtered_visits['initial_zone'] == zone]
    
    if is_admitted is not None:
        filtered_visits = filtered_visits[filtered_visits['is_admitted'] == int(is_admitted)]
    
    if is_lwbs is not None:
        filtered_visits = filtered_visits[filtered_visits['is_lwbs'] == int(is_lwbs)]
    
    # Get case IDs that match filters
    valid_cases = filtered_visits['case_id'].unique()
    
    # Filter event log
    filtered_log = event_log[event_log['case_id'].isin(valid_cases)].copy()
    
    print(f"Filtered to {len(valid_cases):,} visits ({len(filtered_log):,} events)")
    
    return filtered_log

print("✓ filter_event_log() function defined")

✓ filter_event_log() function defined


## Cell 10: Compare DFGs - CTAS 2 (Emergency) vs CTAS 4 (Less Urgent)

In [67]:
# Filter for CTAS 2 (Emergency) patients
print("=" * 50)
print("CTAS 2 - EMERGENCY Patients")
print("=" * 50)
event_log_ctas2 = filter_event_log(event_log, visits, triage_level='2-EMERGENCY')
dfg_ctas2 = build_dfg(event_log_ctas2)
edge_stats_ctas2 = calculate_edge_stats(dfg_ctas2)

print("\nTop 5 Transitions:")
display(edge_stats_ctas2[['from', 'to', 'count', 'median_minutes']].head())

CTAS 2 - EMERGENCY Patients
Filtered to 4,822 visits (29,402 events)

Top 5 Transitions:


,from,to,count,median_minutes
0,Triage,Registration,4326,6.0
1,Discharge,Left ED,4324,11.0
2,Registration,Assessment,3883,36.0
3,Assessment,Discharge,3368,152.0
4,Assessment,Consult Request,1346,136.5


In [68]:
# Filter for CTAS 4 (Less Urgent) patients
print("=" * 50)
print("CTAS 4 - LESS URGENT Patients")
print("=" * 50)
event_log_ctas4 = filter_event_log(event_log, visits, triage_level='4-LESS URGENT')
dfg_ctas4 = build_dfg(event_log_ctas4)
edge_stats_ctas4 = calculate_edge_stats(dfg_ctas4)

print("\nTop 5 Transitions:")
display(edge_stats_ctas4[['from', 'to', 'count', 'median_minutes']].head())

CTAS 4 - LESS URGENT Patients
Filtered to 1,947 visits (9,943 events)

Top 5 Transitions:


,from,to,count,median_minutes
0,Triage,Registration,1915,5.0
1,Discharge,Left ED,1895,5.0
2,Assessment,Discharge,1854,39.0
3,Registration,Assessment,1844,28.0
4,Ambulance Arrival,Triage,93,8.0


## Cell 11: Visualize CTAS 2 vs CTAS 4 Comparison

In [69]:
# Side-by-side comparison: Registration → Assessment (PIA)
print("\n" + "=" * 60)
print("KEY INSIGHT: Registration → Assessment (PIA) Comparison")
print("=" * 60)

pia_ctas2 = edge_stats_ctas2[edge_stats_ctas2['from'] == 'Registration']
pia_ctas2 = pia_ctas2[pia_ctas2['to'] == 'Assessment']

pia_ctas4 = edge_stats_ctas4[edge_stats_ctas4['from'] == 'Registration']
pia_ctas4 = pia_ctas4[pia_ctas4['to'] == 'Assessment']

print(f"\nCTAS 2 (Emergency):")
print(f"  • Median PIA: {pia_ctas2['median_minutes'].values[0]:.0f} minutes")
print(f"  • Target: 15 minutes")
print(f"  • Gap: {pia_ctas2['median_minutes'].values[0] - 15:.0f} minutes over target")

print(f"\nCTAS 4 (Less Urgent):")
print(f"  • Median PIA: {pia_ctas4['median_minutes'].values[0]:.0f} minutes")
print(f"  • Target: 60 minutes")
print(f"  • Status: {'✓ Within target' if pia_ctas4['median_minutes'].values[0] <= 60 else '✗ Over target'}")

# Check for priority inversion
if pia_ctas4['median_minutes'].values[0] < pia_ctas2['median_minutes'].values[0]:
    print("\n⚠️  PRIORITY INVERSION DETECTED!")
    print("    Less urgent patients are being seen faster than emergency patients.")


KEY INSIGHT: Registration → Assessment (PIA) Comparison

CTAS 2 (Emergency):
  • Median PIA: 36 minutes
  • Target: 15 minutes
  • Gap: 21 minutes over target

CTAS 4 (Less Urgent):
  • Median PIA: 28 minutes
  • Target: 60 minutes
  • Status: ✓ Within target

⚠️  PRIORITY INVERSION DETECTED!
    Less urgent patients are being seen faster than emergency patients.


## Cell 12: Sankey Diagrams for Filtered Groups

In [70]:
# CTAS 2 Sankey
fig_ctas2 = create_sankey_diagram(
    dfg_ctas2, 
    title="Patient Flow: CTAS 2 - EMERGENCY",
    min_edge_count=20
)
fig_ctas2.show()

In [71]:
# CTAS 4 Sankey
fig_ctas4 = create_sankey_diagram(
    dfg_ctas4, 
    title="Patient Flow: CTAS 4 - LESS URGENT",
    min_edge_count=10
)
fig_ctas4.show()

---
## Section 4: Zone-Specific Analysis

Compare Yellow Zone (YZ) vs Green Zone (GZ) - the two busiest zones.

---

## Cell 13: Yellow Zone vs Green Zone

In [72]:
# Yellow Zone Analysis
print("=" * 50)
print("YELLOW ZONE (YZ) Analysis")
print("=" * 50)
event_log_yz = filter_event_log(event_log, visits, zone='YZ')
dfg_yz = build_dfg(event_log_yz)
edge_stats_yz = calculate_edge_stats(dfg_yz)

print("\nTop transitions with median durations:")
display(edge_stats_yz[['from', 'to', 'count', 'median_minutes']].head(8))

YELLOW ZONE (YZ) Analysis
Filtered to 4,232 visits (22,777 events)

Top transitions with median durations:


,from,to,count,median_minutes
0,Triage,Registration,4078,5.0
1,Registration,Assessment,4017,51.0
2,Discharge,Left ED,3988,5.0
3,Assessment,Discharge,3607,126.0
4,Assessment,Consult Request,581,146.0
5,Consult Request,Discharge,333,122.0
6,Ambulance Arrival,Triage,321,7.0
7,Consult Arrival,Discharge,249,58.0


In [73]:
# Green Zone Analysis
print("=" * 50)
print("GREEN ZONE (GZ) Analysis")
print("=" * 50)
event_log_gz = filter_event_log(event_log, visits, zone='GZ')
dfg_gz = build_dfg(event_log_gz)
edge_stats_gz = calculate_edge_stats(dfg_gz)

print("\nTop transitions with median durations:")
display(edge_stats_gz[['from', 'to', 'count', 'median_minutes']].head(8))

GREEN ZONE (GZ) Analysis
Filtered to 4,112 visits (21,025 events)

Top transitions with median durations:


,from,to,count,median_minutes
0,Discharge,Left ED,4053,6.0
1,Triage,Registration,4016,5.0
2,Assessment,Discharge,3994,42.0
3,Registration,Assessment,3979,27.0
4,Ambulance Arrival,Triage,187,8.0
5,Registration,Ambulance Transfer,104,1.0
6,Ambulance Transfer,Assessment,103,35.0
7,Assessment,Consult Request,90,56.0


In [74]:
# Compare PIA between zones
print("\n" + "=" * 60)
print("ZONE COMPARISON: Registration → Assessment (PIA)")
print("=" * 60)

def get_pia_stats(edge_stats):
    pia = edge_stats[(edge_stats['from'] == 'Registration') & (edge_stats['to'] == 'Assessment')]
    if len(pia) > 0:
        return pia.iloc[0]
    return None

zones_to_compare = ['Resus', 'YZ', 'GZ', 'EPZ', 'A', 'SA']
zone_comparison = []

for zone in zones_to_compare:
    filtered = filter_event_log(event_log, visits, zone=zone)
    if len(filtered) > 0:
        dfg_zone = build_dfg(filtered)
        stats = calculate_edge_stats(dfg_zone)
        pia = get_pia_stats(stats)
        if pia is not None:
            zone_comparison.append({
                'Zone': zone,
                'Cases': pia['count'],
                'Median PIA (min)': round(pia['median_minutes'], 1),
                'Mean PIA (min)': round(pia['mean_minutes'], 1),
                'Max PIA (min)': round(pia['max_minutes'], 1)
            })

zone_df = pd.DataFrame(zone_comparison)
zone_df = zone_df.sort_values('Median PIA (min)', ascending=False)
print("\n")
display(zone_df)


ZONE COMPARISON: Registration → Assessment (PIA)
Filtered to 421 visits (3,076 events)
Filtered to 4,232 visits (22,777 events)
Filtered to 4,112 visits (21,025 events)
Filtered to 2,503 visits (13,280 events)
Filtered to 3,193 visits (20,691 events)
Filtered to 1,078 visits (7,434 events)




,Zone,Cases,Median PIA (min),Mean PIA (min),Max PIA (min)
1,YZ,4017,51.0,62.6,509.0
4,A,2294,41.0,54.6,591.0
5,SA,715,41.0,53.6,305.0
2,GZ,3979,27.0,35.9,218.0
3,EPZ,2295,25.0,36.5,307.0
0,Resus,249,9.0,13.5,492.0


## Cell 14: Bottleneck Bar Chart

In [75]:
# Visualize zone PIA comparison
fig_zone_pia = px.bar(
    zone_df.sort_values('Median PIA (min)'),
    x='Median PIA (min)',
    y='Zone',
    orientation='h',
    title='Median PIA Time by Zone (Bottleneck Analysis)',
    color='Median PIA (min)',
    color_continuous_scale='RdYlGn_r',  # Red = bad, Green = good
    text='Median PIA (min)'
)

fig_zone_pia.update_traces(textposition='outside')
fig_zone_pia.update_layout(
    height=400,
    showlegend=False,
    xaxis_title='Median PIA Time (minutes)',
    yaxis_title='Zone'
)

fig_zone_pia.show()

---
## Section 5: Identify the "Happy Path" and Deviations

**Happy Path:** The most common, expected patient journey
**Deviations:** Unusual paths that may indicate problems or special cases

---

## Cell 15: Extract Patient Traces

In [76]:
def extract_traces(event_log: pd.DataFrame) -> pd.DataFrame:
    """
    Extract the complete activity sequence (trace) for each case.
    
    WHY: To identify:
    - Happy Path (most common trace)
    - Deviations (unusual traces)
    - Missing steps (skipped Triage/Registration)
    
    Returns:
    --------
    DataFrame with case_id, trace (as string), trace_length
    """
    
    traces = []
    
    for case_id, group in event_log.groupby('case_id'):
        events = group.sort_values('timestamp')
        trace = ' → '.join(events['activity'].tolist())
        
        traces.append({
            'case_id': case_id,
            'trace': trace,
            'trace_length': len(events)
        })
    
    return pd.DataFrame(traces)

# Extract all traces
traces_df = extract_traces(event_log)

# Count trace frequencies
trace_counts = traces_df['trace'].value_counts().reset_index()
trace_counts.columns = ['trace', 'count']
trace_counts['percentage'] = (trace_counts['count'] / len(traces_df) * 100).round(2)

print("=" * 70)
print("TOP 10 PATIENT JOURNEYS (Most Common Traces)")
print("=" * 70)
print("\n")

for i, row in trace_counts.head(10).iterrows():
    print(f"{i+1}. ({row['count']:,} cases, {row['percentage']}%)")
    print(f"   {row['trace']}")
    print()

TOP 10 PATIENT JOURNEYS (Most Common Traces)


1. (11,067 cases, 69.12%)
   Triage → Registration → Assessment → Discharge → Left ED

2. (1,072 cases, 6.7%)
   Ambulance Arrival → Triage → Registration → Ambulance Transfer → Assessment → Discharge → Left ED

3. (759 cases, 4.74%)
   Triage → Registration → Assessment → Consult Request → Consult Arrival → Discharge → Left ED

4. (678 cases, 4.23%)
   Ambulance Arrival → Triage → Ambulance Transfer → Registration → Assessment → Discharge → Left ED

5. (434 cases, 2.71%)
   Ambulance Arrival → Triage → Registration → Ambulance Transfer → Assessment → Consult Request → Consult Arrival → Discharge → Left ED

6. (425 cases, 2.65%)
   Triage → Registration → Assessment → Consult Request → Discharge → Consult Arrival → Left ED

7. (284 cases, 1.77%)
   Triage → Registration → Assessment → Consult Request → Discharge → Left ED

8. (250 cases, 1.56%)
   Ambulance Arrival → Triage → Registration → Ambulance Transfer → Assessment → Consult Request

## Cell 16: Identify Happy Path

In [77]:
# The Happy Path is the most common trace
happy_path = trace_counts.iloc[0]['trace']
happy_path_count = trace_counts.iloc[0]['count']
happy_path_pct = trace_counts.iloc[0]['percentage']

print("=" * 70)
print("✓ HAPPY PATH IDENTIFIED")
print("=" * 70)
print(f"\n{happy_path}")
print(f"\n• {happy_path_count:,} patients ({happy_path_pct}%) follow this exact path")
print(f"• This is the 'standard' patient journey at NTH-ED")

✓ HAPPY PATH IDENTIFIED

Triage → Registration → Assessment → Discharge → Left ED

• 11,067 patients (69.12%) follow this exact path
• This is the 'standard' patient journey at NTH-ED


## Cell 17: Identify Deviations

In [78]:
def identify_deviations(traces_df: pd.DataFrame) -> dict:
    """
    Identify specific types of deviations from standard protocol.
    
    Deviation types:
    1. Missing Triage - Patient bypassed triage
    2. Missing Registration - Skipped registration
    3. Missing Assessment - Never saw a physician (LWBS likely)
    4. Discharge before Assessment - Data error or special case
    5. Has Consult - Required specialist (longer stay expected)
    """
    
    deviations = {
        'missing_triage': [],
        'missing_registration': [],
        'missing_assessment': [],
        'discharge_before_assessment': [],
        'has_consult': [],
        'ambulance_cases': []
    }
    
    for _, row in traces_df.iterrows():
        trace = row['trace']
        case_id = row['case_id']
        
        # Check for missing steps
        if 'Triage' not in trace:
            deviations['missing_triage'].append(case_id)
        
        if 'Registration' not in trace:
            deviations['missing_registration'].append(case_id)
        
        if 'Assessment' not in trace:
            deviations['missing_assessment'].append(case_id)
        
        # Check for out-of-order events
        if 'Discharge' in trace and 'Assessment' in trace:
            if trace.find('Discharge') < trace.find('Assessment'):
                deviations['discharge_before_assessment'].append(case_id)
        
        # Check for consults
        if 'Consult Request' in trace:
            deviations['has_consult'].append(case_id)
        
        # Check for ambulance
        if 'Ambulance Arrival' in trace:
            deviations['ambulance_cases'].append(case_id)
    
    return deviations

# Identify deviations
deviations = identify_deviations(traces_df)

print("=" * 70)
print("DEVIATION ANALYSIS")
print("=" * 70)
print(f"\nTotal visits analyzed: {len(traces_df):,}")
print("\nDeviation Counts:")
print(f"  • Missing Triage:              {len(deviations['missing_triage']):,} cases")
print(f"  • Missing Registration:        {len(deviations['missing_registration']):,} cases")
print(f"  • Missing Assessment:          {len(deviations['missing_assessment']):,} cases")
print(f"  • Discharge before Assessment: {len(deviations['discharge_before_assessment']):,} cases")
print(f"  • Has Consult Request:         {len(deviations['has_consult']):,} cases")
print(f"  • Ambulance Arrivals:          {len(deviations['ambulance_cases']):,} cases")

DEVIATION ANALYSIS

Total visits analyzed: 16,011

Deviation Counts:
  • Missing Triage:              1 cases
  • Missing Registration:        0 cases
  • Missing Assessment:          163 cases
  • Discharge before Assessment: 3 cases
  • Has Consult Request:         2,718 cases
  • Ambulance Arrivals:          3,115 cases


## Cell 18: Deviation Summary Visualization

In [79]:
# Create deviation summary chart
deviation_data = pd.DataFrame({
    'Deviation Type': [
        'Missing Assessment',
        'Missing Registration', 
        'Missing Triage',
        'Discharge Before Assessment'
    ],
    'Count': [
        len(deviations['missing_assessment']),
        len(deviations['missing_registration']),
        len(deviations['missing_triage']),
        len(deviations['discharge_before_assessment'])
    ]
})

deviation_data['Percentage'] = (deviation_data['Count'] / len(traces_df) * 100).round(2)

fig_dev = px.bar(
    deviation_data.sort_values('Count'),
    x='Count',
    y='Deviation Type',
    orientation='h',
    title='Protocol Deviations (Cases Not Following Standard Path)',
    color='Count',
    color_continuous_scale='Reds',
    text='Count'
)

fig_dev.update_traces(textposition='outside')
fig_dev.update_layout(
    height=350,
    showlegend=False,
    xaxis_title='Number of Cases'
)

fig_dev.show()

---
## Section 6: Summary Functions for Dashboard Integration

These functions will be used in the Streamlit dashboard.

---

## Cell 19: Export Functions for Dashboard

In [80]:
# Summary of all process discovery functions

PROCESS_DISCOVERY_FUNCTIONS = """
=================================================================
MODULE 2: PROCESS DISCOVERY - FUNCTION REFERENCE
=================================================================

CORE FUNCTIONS:
---------------
1. build_dfg(event_log) 
   → Returns dict with edges, nodes, edge_durations
   → Use for: Creating process graphs

2. calculate_edge_stats(dfg)
   → Returns DataFrame with transition statistics
   → Use for: Bottleneck analysis, duration tables

3. filter_event_log(event_log, visits, triage_level, zone, ...)
   → Returns filtered event log
   → Use for: Segment-specific analysis

4. extract_traces(event_log)
   → Returns DataFrame with case traces
   → Use for: Happy path, deviation detection

5. identify_deviations(traces_df)
   → Returns dict of deviation types and case IDs
   → Use for: Conformance checking

VISUALIZATION FUNCTIONS:
------------------------
1. create_sankey_diagram(dfg, title, min_edge_count)
   → Returns Plotly Figure
   → Use for: Interactive flow visualization

=================================================================
"""

print(PROCESS_DISCOVERY_FUNCTIONS)


MODULE 2: PROCESS DISCOVERY - FUNCTION REFERENCE

CORE FUNCTIONS:
---------------
1. build_dfg(event_log) 
   → Returns dict with edges, nodes, edge_durations
   → Use for: Creating process graphs

2. calculate_edge_stats(dfg)
   → Returns DataFrame with transition statistics
   → Use for: Bottleneck analysis, duration tables

3. filter_event_log(event_log, visits, triage_level, zone, ...)
   → Returns filtered event log
   → Use for: Segment-specific analysis

4. extract_traces(event_log)
   → Returns DataFrame with case traces
   → Use for: Happy path, deviation detection

5. identify_deviations(traces_df)
   → Returns dict of deviation types and case IDs
   → Use for: Conformance checking

VISUALIZATION FUNCTIONS:
------------------------
1. create_sankey_diagram(dfg, title, min_edge_count)
   → Returns Plotly Figure
   → Use for: Interactive flow visualization




---
## Key Insights for Triage Lead

### Bottlenecks Identified:
1. **Yellow Zone (YZ)** has the longest PIA time - potential staffing issue
2. **Registration → Assessment** is the critical bottleneck across all zones
3. **Results** → Have the fastest PIA (~8 min) - as expected for critical patients

### Priority Concerns:
- Check if CTAS 2 patients are truly prioritized over CTAS 4
- Monitor cases missing Assessment (potential LWBS)

### Action Items:
- Focus resources on YZ zone during peak hours
- Investigate cases with "Discharge before Assessment"
- Track consult delays separately (adds significant time)

---